In [1]:
from nltk.corpus import stopwords
from nltk.tokenize import WordPunctTokenizer
import string
import pandas as pd
import nltk 
import numpy as np
nltk.download('stopwords')
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df_business = pd.read_csv('../data_snapshot/business_snap.csv')

df_review = pd.read_csv('../data_snapshot/review_snap.csv')
df_business.rename(columns={"0":'id', '1':'alias', '2':'name', '3':'url', '4':'review_count', '5':'phone', '6':'distance','7':'latitude','8':'longitude', '9':'rating', '10':'categories', '11':'transactions'}, inplace = True)
df_review.rename(columns={'0':'review_id', '1':'text', '2': 'rating','3':'time_created', '4':'user_id','5':'business_id' }, inplace=True)

In [3]:
df_review.head()

,Unnamed: 0,review_id,text,rating,time_created,user_id,business_id
0,0,1XyJMWQ0fgWszG69vh3INA,Was seated and waited 30 mins without even bei...,1,2023-06-25 11:22:45,WRUIUIVkz7F5IiH7XI8QcA,NaN
1,1,H4FhOjFivi4aMxeddxMPGg,ROASTED CHICKEN & GOAT CHEESE PANINI \n16.50 ~...,5,2023-06-24 19:37:16,y0wA70QDTUeKS_VyLw7A7A,NaN
2,2,qLzO2mwYN3ti8dvAfpxePw,All the vibes!! We dropped in on a Saturday w...,4,2023-04-30 07:04:01,LDoa3UXbySRr0b2pcwrlnw,NaN
3,3,dONpgM2St8gaCZdnaWiTKw,I was so excited about the Food Truck Fest whe...,5,2014-08-16 19:45:05,7DCwwpe524F2AqZqTUGExg,NaN
4,4,V4d7QNZtIbCfMXYcgR5LhA,This is the best Mexican restaurant in Gahanna...,5,2023-05-31 03:41:00,7v5WWyFCONwphGxWqlncEw,NaN


In [4]:
df_business.dropna(inplace=True)
df_business.isna().sum()

Unnamed: 0      0
id              0
alias           0
name            0
url             0
review_count    0
phone           0
distance        0
latitude        0
longitude       0
rating          0
categories      0
transactions    0
dtype: int64

In [5]:
df_business.columns

Index(['Unnamed: 0', 'id', 'alias', 'name', 'url', 'review_count', 'phone',
       'distance', 'latitude', 'longitude', 'rating', 'categories',
       'transactions'],
      dtype='object')

In [10]:
review_user = df_review[['business_id','text','rating','user_id']]
#review_user = review_user.dropna(inplace=True)
review_user = review_user[~review_user['business_id'].isnull()]
review_user

,business_id,text,rating,user_id
42,35VgbRQWRDvQrIku3OfqKA,Used to go here all the time before the new ow...,4,tYT1kCLO8qEkEUFGClQQRg
43,35VgbRQWRDvQrIku3OfqKA,"We do love this local place. Today, we went fo...",5,cAJJC0U8s22Wm3aPROw5SQ
44,35VgbRQWRDvQrIku3OfqKA,"Okay for lunch, typical bar food. Server was s...",4,k4biYB5Tavw-dwn-6tVbmA
45,pW6jIK9Vu18vXigPTfNwUQ,We went here for taco and tequila fest. Paid $...,3,Wu1hrty56b_dZDfLBaXJRg
46,pW6jIK9Vu18vXigPTfNwUQ,Absolute trash. Go if you want to get assaulte...,1,WKllrAlPeSjX-ZM-XofDzA
47,pW6jIK9Vu18vXigPTfNwUQ,"Oh, Park Street Cantina, I don't even know whe...",3,_NpJZ0q8KVI-d2YLL_VpCA
48,BDe-iiPm9Gb7B2qCzlQWCg,Excellent customer service! This is my wife's ...,5,7DCwwpe524F2AqZqTUGExg
49,BDe-iiPm9Gb7B2qCzlQWCg,I ordered the strawberry cake for a birthday p...,5,gxOLcPG1VqMmCPDTWxZ5KQ
50,BDe-iiPm9Gb7B2qCzlQWCg,Service is poor (young lady didn't listen well...,4,Q4LI1r0RWbpYWgvnhIYqRQ
51,CzVVb1PRNRlYk7PDvw8xSg,With new management breathing fresh air into t...,5,Wx4ccSvTc2GG5aYbosaokA


In [11]:
stop = []
for word in stopwords.words('english'):
    s = [char for char in word if char not in string.punctuation]
    stop.append(' '.join(s))
print(stop)

['i', 'm e', 'm y', 'm y s e l f', 'w e', 'o u r', 'o u r s', 'o u r s e l v e s', 'y o u', 'y o u r e', 'y o u v e', 'y o u l l', 'y o u d', 'y o u r', 'y o u r s', 'y o u r s e l f', 'y o u r s e l v e s', 'h e', 'h i m', 'h i s', 'h i m s e l f', 's h e', 's h e s', 'h e r', 'h e r s', 'h e r s e l f', 'i t', 'i t s', 'i t s', 'i t s e l f', 't h e y', 't h e m', 't h e i r', 't h e i r s', 't h e m s e l v e s', 'w h a t', 'w h i c h', 'w h o', 'w h o m', 't h i s', 't h a t', 't h a t l l', 't h e s e', 't h o s e', 'a m', 'i s', 'a r e', 'w a s', 'w e r e', 'b e', 'b e e n', 'b e i n g', 'h a v e', 'h a s', 'h a d', 'h a v i n g', 'd o', 'd o e s', 'd i d', 'd o i n g', 'a', 'a n', 't h e', 'a n d', 'b u t', 'i f', 'o r', 'b e c a u s e', 'a s', 'u n t i l', 'w h i l e', 'o f', 'a t', 'b y', 'f o r', 'w i t h', 'a b o u t', 'a g a i n s t', 'b e t w e e n', 'i n t o', 't h r o u g h', 'd u r i n g', 'b e f o r e', 'a f t e r', 'a b o v e', 'b e l o w', 't o', 'f r o m', 'u p', 'd

In [12]:
def process(mess):
    non_punc = [char for char in mess if char not in string.punctuation]
    non_punc = ''.join(non_punc)
    return " ".join([word for word in non_punc.split() if word.lower() not in stop])

In [13]:
user_text = review_user[['user_id', 'text']]
business_text = review_user[['business_id', 'text']]

In [14]:
user_text = user_text.groupby('user_id').agg({'text': " ".join})
business_text = business_text.groupby('business_id').agg({'text': " ".join})

In [15]:
userid_vectorizer = TfidfVectorizer(tokenizer = WordPunctTokenizer().tokenize, max_features=5000)
userid_vectors = userid_vectorizer.fit_transform(user_text['text'])
userid_vectors.shape

c:\Users\admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


(28, 411)

In [16]:
businessid_vectorizer = TfidfVectorizer(tokenizer = WordPunctTokenizer().tokenize, max_features=5000)
businessid_vectors = businessid_vectorizer.fit_transform(business_text['text'])
businessid_vectors.shape

(10, 411)

In [17]:
userid_rating_matrix = pd.pivot_table(review_user, values='rating', index=['user_id'], columns=['business_id'])
userid_rating_matrix.shape

(28, 10)

In [18]:
P = pd.DataFrame(userid_vectors.toarray(), index=user_text.index, columns=userid_vectorizer.get_feature_names_out())
Q = pd.DataFrame(businessid_vectors.toarray(), index=business_text.index, columns=businessid_vectorizer.get_feature_names_out())

In [19]:
def matrix_factorization(R, P, Q, steps=25, gamma=0.001,lamda=0.02):
    for step in range(steps):
        for i in R.index:
            for j in R.columns:
                if R.loc[i,j]>0:
                    eij=R.loc[i,j]-np.dot(P.loc[i],Q.loc[j])
                    P.loc[i]=P.loc[i]+gamma*(eij*Q.loc[j]-lamda*P.loc[i])
                    Q.loc[j]=Q.loc[j]+gamma*(eij*P.loc[i]-lamda*Q.loc[j])
        e=0
        for i in R.index:
            for j in R.columns:
                if R.loc[i,j]>0:
                    e= e + pow(R.loc[i,j]-np.dot(P.loc[i],Q.loc[j]),2)+lamda*(pow(np.linalg.norm(P.loc[i]),2)+pow(np.linalg.norm(Q.loc[j]),2))
        if e<0.001:
            break
    print(f"Step:{step}")
    return P,Q

In [20]:
%%time
P, Q = matrix_factorization(userid_rating_matrix, P, Q, steps=25, gamma=0.001,lamda=0.02)

Step:24
CPU times: total: 766 ms
Wall time: 783 ms


In [34]:
words = "i want to have dinner with beautiful views"
test_df= pd.DataFrame([words], columns=['text'])
test_df['text'] = test_df['text'].apply(process)
test_vectors = userid_vectorizer.transform(test_df['text'])
test_v_df = pd.DataFrame(test_vectors.toarray(), index=test_df.index, columns=userid_vectorizer.get_feature_names_out())

predictItemRating=pd.DataFrame(np.dot(test_v_df.loc[0],Q.T),index=Q.index,columns=['Rating'])
topRecommendations=pd.DataFrame.sort_values(predictItemRating,['Rating'],ascending=[0])[:7]

#print(df_business[df_business['id'] == 'KJNWztNYuNysB1MVsX0FOA'])
for i in topRecommendations.index:
    print(df_business[df_business['id']==i]['name'])
#    print(df_business[df_business['id']==i]['categories'])
#    print(str(df_business[df_business['id']==i]['rating'])+ ' '+str(df_business[df_business['id']==i]['review_count']))
#    print('')

509    ParkStreetCantina
Name: name, dtype: object
Series([], Name: name, dtype: object)
Series([], Name: name, dtype: object)
Series([], Name: name, dtype: object)
120    YinYueRestaurant
Name: name, dtype: object
Series([], Name: name, dtype: object)
Series([], Name: name, dtype: object)


In [35]:
import pickle
output = open('yelp_recommendation_model_8.pkl', 'wb')
pickle.dump(P,output)
pickle.dump(Q,output)
pickle.dump(userid_vectorizer,output)
output.close()